In [21]:
import pandas as pd 
import numpy as np 
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

# Precisamos fazer isso para importar variávies de outro notebook.
%store -r credits
%store -r movies

C:\Users\renan\AppData\Roaming\Python\Python312\site-packages\IPython\extensions\storemagic.py:148: UserWarning: This is now an optional IPython functionality, using autorestore/credits requires you to install the `pickleshare` library.
  obj = db["autorestore/" + arg]
C:\Users\renan\AppData\Roaming\Python\Python312\site-packages\IPython\extensions\storemagic.py:148: UserWarning: This is now an optional IPython functionality, using autorestore/movies requires you to install the `pickleshare` library.
  obj = db["autorestore/" + arg]


In [22]:
df1=pd.read_csv(r"tmdb_5000_credits.csv",encoding='utf-8', sep=',')
df2=pd.read_csv(r"tmdb_5000_movies.csv",encoding='utf-8', sep=',')

df1.columns = ['id','tittle','cast','crew']
dados = df2.merge(df1,on='id')

# Content Based Filtering

Neste sistema de recomendação, o conteúdo do filme (sinopse, elenco, equipe, palavra-chave, slogan etc.) é usado para encontrar sua semelhança com outros filmes. Em seguida, os filmes que são mais provavelmente semelhantes são recomendados.

In [23]:
dados['overview'].head(5)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

In [24]:
# Inicializando o objeto Vetorizador TF-IDF com a remoção de stopwords em inglês
vetorizador_tfidf = TfidfVectorizer(stop_words='english')

# Preenchendo valores nulos na coluna 'visão geral' com uma string vazia
dados['overview'] = dados['overview'].fillna('')

# Criando uma matriz TF-IDF a partir dos textos da coluna 'visão_geral'
matriz_tfidf = vetorizador_tfidf.fit_transform(dados['overview'])

# Extraindo a forma (shape) da matriz TF-IDF
# Essa linha retorna uma tupla representando o número de linhas (documentos) e colunas (features) na matriz TF-IDF
print("Forma da Matriz TF-IDF:", matriz_tfidf.shape)

Forma da Matriz TF-IDF: (4803, 20978)


In [25]:
# Calculando a similaridade cosseno entre as linhas da matriz TF-IDF
similaridade_cosseno = linear_kernel(matriz_tfidf, matriz_tfidf)

# Criando uma série de índices onde os índices são os títulos dos documentos e os valores são os índices correspondentes no DataFrame
indices = pd.Series(dados.index, index=dados['title']).drop_duplicates()

In [26]:
def obter_recomendacoes(titulo, similaridade_cosseno=similaridade_cosseno):
    # Obtendo o índice do título fornecido
    indice = indices[titulo]

    # Obtendo os scores de similaridade cosseno para todos os filmes
    scores_similares = list(enumerate(similaridade_cosseno[indice]))

    # Ordenando os scores de similaridade em ordem decrescente
    scores_similares = sorted(scores_similares, key=lambda x: x[1], reverse=True)

    # Excluindo o próprio filme (índice 0)
    scores_similares = scores_similares[1:11]

    # Obtendo os índices dos filmes recomendados
    indices_filmes_recomendados = [i[0] for i in scores_similares]

    # Retornando os títulos dos filmes recomendados
    return dados['title'].iloc[indices_filmes_recomendados]

In [27]:
obter_recomendacoes('Avatar')

3604                       Apollo 18
2130                    The American
634                       The Matrix
1341            The Inhabited Island
529                 Tears of the Sun
1610                           Hanna
311     The Adventures of Pluto Nash
847                         Semi-Pro
775                        Supernova
2628             Blood and Chocolate
Name: title, dtype: object

In [28]:
obter_recomendacoes('The Avengers')

7               Avengers: Age of Ultron
3144                            Plastic
1715                            Timecop
4124                 This Thing of Ours
3311              Thank You for Smoking
3033                      The Corruptor
588     Wall Street: Money Never Sleeps
2136         Team America: World Police
1468                       The Fountain
1286                        Snowpiercer
Name: title, dtype: object